In [0]:
import torch
import pandas as pd
import tqdm.notebook as tqdm

In [66]:
df = pd.read_csv('smile-annotations-final.csv',
                 names = ['id', 'text', 'category'])
df.category.unique()

array(['nocode', 'happy', 'not-relevant', 'angry', 'disgust|angry',
       'disgust', 'happy|surprise', 'sad', 'surprise', 'happy|sad',
       'sad|disgust', 'sad|angry', 'sad|disgust|angry'], dtype=object)

In [67]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']
df.category.unique()

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [68]:
possible_labels = df.category.unique()
label_dict = {}
for index,possible_labels in enumerate(possible_labels):
  label_dict[possible_labels] = index;
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [0]:
df['label'] = df.category.replace(label_dict)

In [70]:
df

,id,text,category,label
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
5,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
...,...,...,...,...
3078,611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
3079,612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
3080,613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0
3082,615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy,0


now part 3

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_val,y_train,y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.13,
    random_state = 17,
    stratify = df.label.values
)
df['data_type'] = ['not_set']*df.shape[0]


In [73]:
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'
df.groupby(['category', 'label', 'data_type']).count()

id  text
category     label data_type           
angry        2     train       50    50
                   val          7     7
disgust      3     train        5     5
                   val          1     1
happy        0     train      989   989
                   val        148   148
not-relevant 1     train      186   186
                   val         28    28
sad          4     train       28    28
                   val          4     4
surprise     5     train       30    30
                   val          5     5

In [0]:
# !pip install transformers
from transformers import BertTokenizer

In [75]:

from torch.utils.data import TensorDataset
import torch
print(torch.__version__)

1.5.0+cu101


In [0]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [0]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_masks=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_masks=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

In [78]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
label_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_train.size()

torch.Size([1288, 256])

In [79]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
label_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
label_val = torch.tensor(df[df.data_type == 'val'].label.values)
input_ids_train.size()

torch.Size([1288, 256])

In [0]:
dataset_train = TensorDataset(input_ids_train,attention_masks_train,label_train)
dataset_val = TensorDataset(input_ids_val,attention_masks_val,label_val)

part 4 now

In [0]:
from transformers import BertForSequenceClassification

In [0]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

data loaders

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [0]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [0]:
optimizer  = AdamW(
    model.parameters(),
    lr=1e-5, 
    eps=1e-8
)
epochs = 10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

lvl up bruh

In [0]:
import numpy as np
from sklearn.metrics import f1_score
def f1_score_func(preds, labels):
    pass
def accuracy_per_class(preds, labels):
    pass

In [0]:
import numpy as np
from sklearn.metrics import f1_score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_trues = labels_flat[labels_flat == label]
        print(f'class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}')

In [98]:

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [0]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [114]:
from tqdm import tqdm
for epoch in tqdm(range(1, epochs+1)):

  model.train()
  loss_train_total = 0

  progress_bar = tqdm(dataloader_train,
                      desc = 'Epoch {:1d}'.format(epoch),
                      leave=False,
                      disable=False
                      )
  for batch in progress_bar:

    model.zero_grad()
    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2]
    }

    outputs = model(**inputs)

    loss = outputs[0]
    loss_train_total += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

  torch.save(model.state_dict(), f'BERT_Bruh{epoch}.model')
  tqdm.write('\nEpoch {epoch}')
  loss_train_avg = loss_train_total/len(dataloader_train)
  tqdm.write(f'Training loss: {loss_train_avg}')

  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 score (weighted): {val_f1}')
  pass




  0%|          | 0/10 [00:00<?, ?it/s]



Epoch 1:   0%|          | 0/41 [00:00<?, ?it/s]



Epoch 1:   0%|          | 0/41 [00:02<?, ?it/s, training_loss=0.083]



Epoch 1:   2%|▏         | 1/41 [00:02<01:40,  2.52s/it, training_loss=0.083]

RuntimeError: ignored

In [0]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

In [0]:
model.to(device)
pass

In [111]:
model.load_state_dict(torch.load('BERT_Bruh9.model',
                                 map_location=torch.device('cuda')))

<All keys matched successfully>

In [0]:
_, predictions, true_vals = evaluate(dataloader_val)

In [113]:
accuracy_per_class(predictions, true_vals)

class: happy
Accuracy: 141
class: not-relevant
Accuracy: 16
class: angry
Accuracy: 4
class: disgust
Accuracy: 0
class: sad
Accuracy: 0
class: surprise
Accuracy: 1


In [119]:
predictions

array([[ 2.5000694 ,  0.14889415, -1.5719748 , -1.0601035 , -0.15449965,
        -0.1107467 ],
       [ 4.6317906 , -0.7091674 , -1.4788564 , -1.2089449 , -1.311177  ,
        -1.3228396 ],
       [-0.706759  ,  3.3103993 , -0.2491186 , -0.8140529 , -0.79440236,
        -0.70977426],
       ...,
       [ 0.5881436 ,  2.8390965 , -0.4658705 , -1.2754145 , -0.6923881 ,
        -1.0819327 ],
       [-1.0995508 ,  0.723639  ,  1.9056774 , -0.28469217, -0.42174035,
        -1.0473379 ],
       [ 4.5232096 , -0.527999  , -1.5181507 , -1.2805042 , -1.4597163 ,
        -1.1265925 ]], dtype=float32)